In [1]:
import torch

In [2]:
torch.cuda.is_available()

True

In [3]:
import torchvision

In [4]:
import torchaudio

In [5]:
import timm

In [6]:
print(torch.version.cuda)

11.7


In [7]:
!nvcc --version

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2022 NVIDIA Corporation
Built on Tue_Mar__8_18:18:20_PST_2022
Cuda compilation tools, release 11.6, V11.6.124
Build cuda_11.6.r11.6/compiler.31057947_0


In [1]:
!pip install audioset-download

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 82.0 MB/s eta 0:00:00:00:01
  Using cached websockets-11.0.3-cp38-cp38-manylinux_2_5_x86_64.manylinux1_x86_64.manylinux_2_17_x86_64.manylinux2014_x86_64.whl (130 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.8/2.8 MB 40.8 MB/s eta 0:00:00ta 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.4/194.4 KB 28.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 74.7 MB/s eta 0:00:00
You should consider upgrading via the '/home/ben2002chou/code/cav-mae/cavmae1017/bin/python -m pip install --upgrade pip' command.


In [3]:
from IPython.display import Audio

# Load the audio file
audio_file_path = '/grand/EVITA/ben/AudioSet/balanced/audio_samples/gFtJiTh6EM8.wav'

# Play the audio file
Audio(audio_file_path)


In [ ]:
ls -l /grand/EVITA/ben/AudioSet/balanced/audio_samples/gFtJiTh6EM8.wav

In [3]:
import torchaudio

In [10]:
import torch

In [4]:
print(torchaudio.get_audio_backend())


sox_io


In [5]:
waveform, sample_rate = torchaudio.load('/grand/EVITA/ben/AudioSet/balanced/audio_samples/gFtJiTh6EM8.wav', format='wav')


In [6]:
print(waveform)

tensor([[-0.0020,  0.0055, -0.0129,  ...,  0.0000,  0.0000,  0.0000]])


In [7]:
print(sample_rate)

16000


In [11]:
if not torch.isfinite(waveform).all():
    raise ValueError("Waveform contains non-finite values.")

In [8]:
try:
    waveform, sample_rate = torchaudio.load('/grand/EVITA/ben/AudioSet/balanced/audio_samples/gFtJiTh6EM8.wav', format='wav')
except Exception as e:
    print(e)


In [12]:
print(waveform.shape)


torch.Size([1, 160768])


In [14]:
    def _wav2fbank(self, filename, filename2=None, mix_lambda=-1):
        try:
            # no mixup
            if filename2 is None:
                waveform, sr = torchaudio.load(filename)
                waveform = waveform - waveform.mean()
            # mixup
            else:
                waveform1, sr = torchaudio.load(filename)
                waveform2, _ = torchaudio.load(filename2)

                waveform1 = waveform1 - waveform1.mean()
                waveform2 = waveform2 - waveform2.mean()

                if waveform1.shape[1] != waveform2.shape[1]:
                    if waveform1.shape[1] > waveform2.shape[1]:
                        # padding
                        temp_wav = torch.zeros(1, waveform1.shape[1])
                        temp_wav[0, 0:waveform2.shape[1]] = waveform2
                        waveform2 = temp_wav
                    else:
                        # cutting
                        waveform2 = waveform2[:, :waveform1.shape[1]]

                mix_waveform = mix_lambda * waveform1 + (1 - mix_lambda) * waveform2
                waveform = mix_waveform - mix_waveform.mean()

            try:
                fbank = torchaudio.compliance.kaldi.fbank(waveform, htk_compat=True, sample_frequency=sr, use_energy=False, window_type='hanning', num_mel_bins=self.melbins, dither=0.0, frame_shift=10)
            except Exception as e:
                fbank = torch.zeros([512, 128]) + 0.01
                logging.error(f'Error in feature extraction: {e}')

            target_length = self.target_length
            n_frames = fbank.shape[0]

            p = target_length - n_frames

            # cut and pad
            if p > 0:
                m = torch.nn.ZeroPad2d((0, 0, 0, p))
                fbank = m(fbank)
            elif p < 0:
                fbank = fbank[:target_length, :]

        except FileNotFoundError:
            logging.error(f'File not found: {filename}')
            return torch.zeros([self.target_length, 128]) + 0.01
        except RuntimeError as e:
            logging.error(f'Runtime error during loading audio: {e}')
            return torch.zeros([self.target_length, 128]) + 0.01
        except Exception as e:
            logging.error(f'An unexpected error occurred: {e}')
            return torch.zeros([self.target_length, 128]) + 0.01
        return fbank

In [15]:
wav2fbank = _wav2fbank('/grand/EVITA/ben/AudioSet/balanced/audio_samples/gFtJiTh6EM8.wav')

NameError: name 'logging' is not defined